In [2]:
from PIL import Image
import cv2 as cv 
import os
import numpy as np
import pandas as pd
import sys

In [3]:
# root_folder = os.getcwd()
sys.path.append(os.path.join(os.getcwd()))
os.environ["SARTORIUS"] = os.path.join(os.getcwd())
sart = os.environ["SARTORIUS"]


In [4]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int)
                       for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


In [5]:
def annotation_conversion(annotation, shape):
    try:
        img_focus = np.zeros(shape)
        img_focus = rle_decode(annotation, shape)
        ycoordinates = np.where(img_focus.sum(axis=1, keepdims=False))
        xcoordinates = np.where(img_focus.sum(axis=0, keepdims=False))
        tuple_dims = (ycoordinates[0][0], ycoordinates[0][-1], xcoordinates[0]
                      [0],    xcoordinates[0][-1])
        return tuple_dims
    except:
        return None


In [6]:
file_path = os.path.join(sart,'sartorius-cell-instance-segmentation', 'train.csv')
df = pd.read_csv(file_path)
shape = (520, 704)

In [7]:
df.head().annotation.apply(lambda x: annotation_conversion(x, shape))

0    (167, 196, 576, 602)
1    (268, 294, 359, 373)
2    (246, 275, 379, 400)
3    (279, 314, 296, 326)
4    (238, 260, 255, 281)
Name: annotation, dtype: object

In [8]:
df['tuple_dims']= df.annotation.apply(lambda x: annotation_conversion(x,shape))

In [9]:
df['new_path'] = df.apply(lambda x: x[0] +'_'+ str(x.name)+'.png', axis=1)


In [10]:
def create_subImage(input_file_path, output_file_path,crop_param,shape = (100,100)):
    try:
        im = cv.imread(input_file_path)
        one_part = im[:, :, 0]
        left, right, top, bottom = crop_param
        r_imaged = cv.resize(one_part[left:right, top:bottom], shape, interpolation=cv.INTER_AREA)
        cv.imwrite(output_file_path, r_imaged)

        return crop_param
    except Exception as E:
        print('Unable to process Image.',str(E))


In [ ]:
df.apply(lambda x: create_subImage(os.path.join(sart,'sartorius-cell-instance-segmentation', 'train', str(x.id)+ '.png'), os.path.join(sart, 'processed_data','train', str(x.new_path)),x.tuple_dims,(100,100)), axis=1)


# Validating the crop Region:

In [ ]:
index=0
annotation = df.annotation[index]


In [ ]:
img2 = rle_decode(annotation, shape)
left, top, right,bottom = annotation_conversion(annotation, shape)


In [ ]:
# reading_image
im= cv.imread(os.path.join(sart,'train',df.id[index]+'.png'))
one_part = im[:,:,0]

In [ ]:
r_imaged = cv.resize(one_part[left:right, top:bottom], (100,100), interpolation=cv.INTER_AREA)


In [ ]:
cv.imwrite("test123.png", r_imaged)
cv.imwrite("test123_2.png", cv.addWeighted(one_part, 0.7, img2*255, 0.1, 0))
one_part[left:right, top:bottom]=255
cv.imwrite("test123_3.png", one_part)


# Subsetting the dataset for Training

In [11]:
def data_pre_processing_image(image_path):
    
    input_feature = cv.imread(image_path)
    return input_feature.reshape(-1)


def data_pre_processing_result(cell_type):
    def label_assignment(cell_type):
        if cell_type == 'shsy5y':
            return np.array([0, 0, 1])
        elif cell_type == 'astro':
            return np.array([1, 0, 0])
        else:
            return np.array([0, 1, 0])

    y_output = label_assignment(cell_type)
    return y_output


In [12]:
df['x']= df.apply(lambda x: data_pre_processing_image(os.path.join(sart, 'processed_data','train', x.new_path)),axis=1 )

In [13]:
df['y']= df.apply(lambda x: data_pre_processing_result(x.cell_type),axis=1 )

In [14]:
X_train = np.stack(df.x,axis=0)

In [15]:
Y_train = np.stack(df.y, axis=0)
np.savetxt('Y_train.csv', Y_train, delimiter=',')


In [16]:
np.savetxt('X_train.csv', X_train, delimiter=',')